# RunnablBranch
- 입력에 따라 동적으로 로직을 분기할 수 있도록 할 수 있다
- 입력 데이터의 다양성과 변동성이 큰 경우에 유용하며, 코드를 간결하게 할 수 있어서 유지보수에도 용이하다
- 분기하는 2가지 방법이 있다
    1. `RunnableLambda`에서 조건부로 실행 (권장)
    1. `RunnableBranch` 사용

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
import os
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_community.chat_models import ChatOllama

llm = HuggingFaceEndpoint(
    repo_id=os.environ["MODEL_ID"], 
    max_new_tokens=2048,
    temperature=0.1,
    huggingfacehub_api_token=os.environ["HF_API_KEY"],
)
model = ChatHuggingFace(llm=llm)

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(
/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dudaji/.cache/huggingface/token
Login successful


/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatHuggingFace` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import ChatHuggingFace`.
  warn_deprecated(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """주어진 사용자 질문을 `수학`, `과학`, 또는 `기타` 중 하나로 분류하세요. 한 단어 이상으로 응답하지 마세요.

<question>
{question}
</question>

Classification:"""
)

# 체인을 생성합니다.
chain = (
    prompt
    | model
    | StrOutputParser()  # 문자열 출력 파서를 사용합니다.
)

In [8]:
print(chain.invoke({"question": "2+2 는 무엇인가요?"}))
print(chain.invoke({"question": "작용 반작용의 법칙은 무엇인가요?"}))
print(chain.invoke({"question": "Google은 어떤 회사인가요?"}))

수학
과학
기타


In [5]:
math_chain = (
    PromptTemplate.from_template(
        """You are an expert in math. \
Always answer questions starting with "깨봉선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | model
)

science_chain = (
    PromptTemplate.from_template(
        """You are an expert in science. \
Always answer questions starting with "아이작 뉴턴 선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | model
)

general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question concisely:

Question: {question}
Answer:"""
    )
    | model
)

In [15]:
# 사용자 정의 함수로 분기하기 (권장)
def route(info):
    # 주제에 "수학"이 포함되어 있는 경우
    if "수학" in info["topic"].lower():
        # datascience_chain을 반환
        return math_chain
    # 주제에 "과학"이 포함되어 있는 경우
    elif "과학" in info["topic"].lower():
        # art_chain을 반환
        return science_chain
    # 그 외의 경우
    else:
        # general_chain을 반환
        return general_chain

In [18]:
from langchain_core.runnables import RunnableLambda

full_chain = (
    {"topic": chain, "question": lambda x: x["question"]}
    | RunnableLambda(
        # 경로를 지정하는 함수를 인자로 전달합니다.
        route
    )
    | StrOutputParser()
)

In [21]:
print(full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."}))
print(full_chain.invoke({"question": "중력은 어떻게 작용하나요?"}))
print(full_chain.invoke({"question": "인간은 어떻게 해야 행복을 느낄까요?"}))

깨봉선생님께서 말씀하시기를... 미적분의 개념은, 함수의 변화율을 측정하는 방법을 말합니다. 일반적으로, 함수의 입력인 독립변수 x가 변화할 때, 함수의 출력인 종속변수 y가 어떻게 변화하는지 분석하는 것입니다. 이 분석을 통해, 함수의 특징을 파악하고, 실제 세계의 현상을 모델링하는 데 사용됩니다.
아이작 뉴턴 선생님께서 말씀하시기를... "Gravity is a force that acts between two objects with mass, and it is proportional to the product of their masses and inversely proportional to the square of the distance between them." (중력은 질량이 있는 두 개의 물체 사이에 작용하는 힘으로, 그 물체의 질량의 곱과 거리의 제곱에 반비례하는 관계에 있습니다.)
인간은 다음과 같은 방법으로 행복을 느낄 수 있습니다.

* 긍정적인 생각과 마음을 가질 것
* 건강하고 안전한 생활을 즐기고
* 좋은 관계를 맺고 있는 것
* 자신의 꿈과 목표를 추구하고
* 감사하고 자비를 가질 것

(Translation: Humans can feel happiness by having positive thoughts and mind, enjoying a healthy and safe life, having good relationships, pursuing their dreams and goals, and being grateful and merciful.)


In [6]:
# RunnableBranch로 분기하기
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "수학" in x["topic"], math_chain),
    (lambda x: "과학" in x["topic"], science_chain),
    general_chain,
)

full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch | StrOutputParser()

In [7]:
print(full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."}))
print(full_chain.invoke({"question": "중력은 어떻게 작용하나요?"}))
print(full_chain.invoke({"question": "인간은 어떻게 해야 행복을 느낄까요?"}))

깨봉선생님께서 말씀하시기를... 미적분의 개념은, 함수의 변화율을 측정하는 방법을 말합니다. 일반적으로, 함수의 입력인 독립변수 x가 변화할 때, 함수의 출력인 종속변수 y가 어떻게 변화하는지 분석하는 것입니다. 이 분석을 통해, 함수의 특징을 파악하고, 실제 세계의 현상을 모델링하는 데 사용됩니다.
아이작 뉴턴 선생님께서 말씀하시기를... "Gravity is a force that acts between two objects with mass, and it is proportional to the product of their masses and inversely proportional to the square of the distance between them." (중력은 질량이 있는 두 개의 물체 사이에 작용하는 힘으로, 그 물체의 질량의 곱과 거리의 제곱에 반비례하는 관계에 있습니다.)
인간은 다음과 같은 방법으로 행복을 느낄 수 있습니다.

* 긍정적인 생각과 마음을 가질 것
* 건강하고 안전한 생활을 즐기고
* 좋은 관계를 맺고 있는 것
* 자신의 꿈과 목표를 추구하고
* 감사하고 자비를 가질 것

(Translation: Humans can feel happiness by having positive thoughts and mind, enjoying a healthy and safe life, having good relationships, pursuing their dreams and goals, and being grateful and merciful.)
